Imports and testing code for index access

In [0]:
%pip install --upgrade databricks-langchain langchain-community langchain databricks-sql-connector

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install --force-reinstall typing-extensions>=4.7
dbutils.library.restartPython()


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:

from databricks_langchain import ChatDatabricks

chat_model = ChatDatabricks(
    # endpoint="databricks-meta-llama-3-3-70b-instruct",
    endpoint="databricks-qwen3-next-80b-a3b-instruct",
    temperature=0.1,
    max_tokens=250,
)
chat_model.invoke("How to use Databricks?")

AIMessage(content='Using **Databricks** involves several key steps, from setting up an account to running data engineering, data science, and machine learning workflows. Here’s a comprehensive, step-by-step guide to help you get started:\n\n---\n\n## ✅ **1. Understand What Databricks Is**\nDatabricks is a cloud-based **unified analytics platform** built on Apache Spark. It enables:\n- **Data Engineering** (ETL, data pipelines)\n- **Data Science & Machine Learning** (notebooks, MLflow, model registry)\n- **Business Intelligence** (dashboards via Databricks SQL)\n- **Real-time analytics** (Delta Lake, Structured Streaming)\n\nIt runs on **AWS, Azure, or GCP**.\n\n---\n\n## ✅ **2. Sign Up for Databricks**\n\n### Option A: Free Trial (Recommended for beginners)\n1. Go to: [https://databricks.com/try-databricks](https://databricks.com/try-databricks)\n2. Fill in your details (name, email, company).\n3. Choose a cloud provider (AWS, Azure, or GCP).\n4. Databricks will provision a **free tria

In [0]:
# from databricks.vector_search.client import VectorSearchClient
# vsc = VectorSearchClient(disable_notice=True)

In [0]:
question = "how to monitor and manage personal access token?"
VECTOR_SEARCH_ENDPOINT_NAME = "databricks_doc_index" 
vs_index_fullname = "workspace.default.db_docs_index"

# results = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).similarity_search(
#   query_text=question,
#   columns=["id", "content"],
#   num_results=1)
# docs = results.get('result', {}).get('data_array', [])
# docs

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


[[22694.0,
  'Monitor and manage personal access tokens  \nTo authenticate to the Databricks REST API, a user can create a personal access token and use it in their REST API request. This article explains how workspace admins can manage personal access tokens in their workspace.  \nTo create a personal access token, see Databricks personal access token authentication.  \nTo create a personal access token on behalf of a service principal, see Manage tokens for a service principal.  \nOverview of personal access token management\nOverview of personal access token management\nPersonal access tokens are enabled by default for all Databricks workspaces that were created in 2018 or later.  \nWhen personal access tokens are enabled on a workspace, users with the CAN USE permission can generate personal access tokens to access Databricks REST APIs, and they can generate these tokens with any expiration date they like, including an indefinite lifetime. By default, no non-admin workspace users h

In [0]:
from databricks_langchain import VectorSearchRetrieverTool, ChatDatabricks

# Initialize the retriever tool.
vs_tool = VectorSearchRetrieverTool(
  endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
  index_name=vs_index_fullname,
  tool_name="databricks_docs_retriever",
  tool_description="Retrieves information about Databricks products from official Databricks documentation.",
  disable_notice=True
)

# Run a query against the vector search index locally for testing
vs_tool.invoke("Explain how Personal Access Token works in databricks")

# Bind the retriever tool to your Langchain LLM of choice
llm = ChatDatabricks(endpoint="databricks-qwen3-next-80b-a3b-instruct")
llm_with_tools = llm.bind_tools([vs_tool])

# Chat with your LLM to test the tool calling functionality
llm_with_tools.invoke("Explain how Personal Access Token works in databricks")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_88256515-c5ed-4304-8ee2-90513524015a', 'function': {'arguments': '{"query": "Personal Access Token in Databricks"}', 'name': 'databricks_docs_retriever'}, 'type': 'function'}]}, response_metadata={'usage': {'prompt_tokens': 200, 'completion_tokens': 31, 'total_tokens': 231}, 'prompt_tokens': 200, 'completion_tokens': 31, 'total_tokens': 231, 'model': 'qwen3-next-instruct-091725', 'model_name': 'qwen3-next-instruct-091725', 'finish_reason': 'tool_calls'}, id='lc_run--019b7e30-0a13-7f31-a85f-bb24747f293f-0', tool_calls=[{'name': 'databricks_docs_retriever', 'args': {'query': 'Personal Access Token in Databricks'}, 'id': 'call_88256515-c5ed-4304-8ee2-90513524015a', 'type': 'tool_call'}])

Trace(trace_id=tr-22e6dc2c37b514cd2f8562470f791575)

In [0]:
%sql
CREATE OR REPLACE FUNCTION main.default.databricks_docs_vector_search (
  -- The agent uses this comment to determine how to generate the query string parameter.
  query STRING
  COMMENT 'The query string for searching Databricks documentation.'
) RETURNS TABLE
-- The agent uses this comment to determine when to call this tool. It describes the types of documents and information contained within the index.
COMMENT 'Executes a search on Databricks documentation to retrieve text documents most relevant to the input query.' RETURN
SELECT
  content as content,
  map('uri', 'id') as metadata
FROM
  vector_search(
    -- Specify your Vector Search index name here
    index => 'workspace.default.db_docs_index',
    -- workspace.default.db_docs_index
    query => query,
    num_results => 5
  )

In [0]:
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(
    function_names=[
        "main.default.databricks_docs_vector_search"
    ]
)
tools = toolkit.tools

In [0]:
## Test unity catalog tool

print(tools[0])
tools[0].invoke({"query":"how to monitor personal access token in databricks?"})

name='main__default__databricks_docs_vector_search' description='Executes a search on Databricks documentation to retrieve text documents most relevant to the input query.' args_schema=<class 'unitycatalog.ai.core.utils.function_processing_utils.main__default__databricks_docs_vector_search__params'> func=<function UCFunctionToolkit.uc_function_to_langchain_tool.<locals>.func at 0xffeb1f06d260> uc_function_name='main.default.databricks_docs_vector_search' client_config={'profile': None}


'{"format": "CSV", "value": "content,metadata\\n\\"Databricks personal access token authentication  \\nDatabricks personal access tokens are one of the most well-supported types of credentials for resources and operations at the Databricks workspace level. Many storage mechanisms for credentials and related information, such as environment variables and Databricks configuration profiles, provide support for Databricks personal access tokens. Although users can have multiple personal access tokens in a Databricks workspace, each personal access token works for only a single Databricks workspace. The number of personal access tokens per user is limited to 600 per workspace.  \\nNote  \\nTo automate Databricks account-level functionality, you cannot use Databricks personal access tokens. Instead, you must use either OAuth tokens for Databricks account admin users or service principals, or the username and password of Databricks account-level admins. For more information, see:  \\nBasic au

In [0]:
# import mlflow
import yaml, sys, os
import mlflow.models
# Add the ../agent_eval path relative to current working directory
# agent_eval_path = os.path.abspath(os.path.join(os.getcwd(), "../02-agent-eval"))
conf_path = os.path.abspath(os.path.join(os.getcwd(), "agent_config.yaml"))

sys.path.append(agent_conf_path)
# Let's also use the same experiment as in our previous notebook to keep all the trace in a single place
# mlflow.set_experiment(agent_eval_path+"/02.1_agent_evaluation")
# conf_path = os.path.join(agent_eval_path, 'agent_config.yaml')

try:
    config = yaml.safe_load(open(conf_path))
    config["config_version_name"] = "model_with_retriever"
    config["retriever_config"] =  {
        "index_name": vs_index_fullname,
        "tool_name": "product_technical_docs_retriever",
        "num_results": 1,
        "description": "Retrieves internal documentation about our products, infrastructure, router and other, including features, usage, and troubleshooting. Use this tool for any questions about product documentation or product issues."
    }
    yaml.dump(config, open(conf_path, "w"))
except Exception as e:
    print(f"Skipped update - ignore for job run - {e}")

model_config = mlflow.models.ModelConfig(development_config=conf_path)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can